In [5]:
import json
import regex as re

from ast import literal_eval

import pandas as pd

import dtale

pd.set_option('display.max_columns', None) # display all columns in DF

# Data from OpenFDA NDC

In [6]:
with open('raw_openFDA_org_NDC_data.json', 'r') as f: # reading download json file
    data = json.load(f)
    
df_openFDA_NDC_org = pd.DataFrame(data['results']) # creating a DataFrame

print(len(df_openFDA_NDC_org))

130115


In [7]:
df_openFDA_NDC_org.head()

,product_ndc,generic_name,labeler_name,brand_name,finished,packaging,listing_expiration_date,openfda,marketing_category,dosage_form,spl_id,product_type,marketing_start_date,product_id,application_number,brand_name_base,marketing_end_date,active_ingredients,route,pharm_class,brand_name_suffix,dea_schedule
0,72835-601,Mometasone Furoate and Dimethicone,"V2 Pharma, LLC",Quinixil,True,"[{'package_ndc': '72835-601-02', 'description'...",20231231,"{'manufacturer_name': ['V2 Pharma, LLC'], 'rxc...",ANDA,KIT,d371387b-e6d5-4e49-a489-57ad96af442a,HUMAN PRESCRIPTION DRUG,20190603,72835-601_d371387b-e6d5-4e49-a489-57ad96af442a,ANDA076216,Quinixil,NaN,NaN,NaN,NaN,NaN,NaN
1,0472-0735,Miconazole Nitrate,"Actavis Pharma, Inc.",Antifungal,True,"[{'package_ndc': '0472-0735-14', 'description'...",NaN,"{'manufacturer_name': ['Actavis Pharma, Inc.']...",OTC MONOGRAPH FINAL,CREAM,e344ee5c-3151-48e3-a5d4-f48011aed57b,HUMAN OTC DRUG,19970601,0472-0735_e344ee5c-3151-48e3-a5d4-f48011aed57b,part333C,Antifungal,20221130,"[{'name': 'MICONAZOLE NITRATE', 'strength': '2...",[TOPICAL],"[Azole Antifungal [EPC], Azoles [CS]]",NaN,NaN
2,72839-961,Miconazole Nitrate 2%,"Derma Care Research Labs, LLC",Warrior Antifungal Foot,True,"[{'package_ndc': '72839-961-02', 'description'...",20221231,{'manufacturer_name': ['Derma Care Research La...,OTC MONOGRAPH FINAL,POWDER,cf2e9dc1-275c-4454-e053-2995a90af295,HUMAN OTC DRUG,20210202,72839-961_cf2e9dc1-275c-4454-e053-2995a90af295,part333C,Warrior Antifungal Foot,NaN,"[{'name': 'MICONAZOLE NITRATE', 'strength': '2...",[TOPICAL],"[Azole Antifungal [EPC], Azoles [CS]]",NaN,NaN
3,0472-0804,desonide,"Actavis Pharma, Inc.",Desonide,True,"[{'package_ndc': '0472-0804-15', 'description'...",20221231,"{'manufacturer_name': ['Actavis Pharma, Inc.']...",NDA AUTHORIZED GENERIC,CREAM,8d3633e6-80d9-4779-862a-b608180e202a,HUMAN PRESCRIPTION DRUG,20130801,0472-0804_8d3633e6-80d9-4779-862a-b608180e202a,NDA019048,Desonide,NaN,"[{'name': 'DESONIDE', 'strength': '.5 mg/g'}]",[TOPICAL],[Corticosteroid Hormone Receptor Agonists [MoA...,NaN,NaN
4,72843-461,DESMOPRESSIN ACETATE,UBI Pharma Inc.,DESMOPRESSIN ACETATE,True,"[{'package_ndc': '72843-461-10', 'description'...",20221231,"{'manufacturer_name': ['UBI Pharma Inc.'], 'rx...",ANDA,INJECTION,b01c1ff8-764d-360e-e053-2995a90aa0bd,HUMAN PRESCRIPTION DRUG,20200917,72843-461_b01c1ff8-764d-360e-e053-2995a90aa0bd,ANDA210223,DESMOPRESSIN ACETATE,NaN,"[{'name': 'DESMOPRESSIN ACETATE', 'strength': ...","[INTRAVENOUS, SUBCUTANEOUS]","[Factor VIII Activator [EPC], Increased Coagul...",NaN,NaN


Removing unwished data

Removing all non human prescription drugs and drugs of the type: EMERGENCY USE AUTHORIZATION, UNAPPROVED DRUG FOR USE IN DRUG SHORTAGE, UNAPPROVED DRUG OTHER. ALso removing not finished drugs as well as marketing category: UNAPPROVED MEDICAL GAS, DRUG FOR FURTHER PROCESSING and UNAPPROVED HOMEOPATHIC

In [9]:
df_openFDA_NDC = df_openFDA_NDC_org
df_openFDA_NDC = df_openFDA_NDC.loc[df_openFDA_NDC['finished'] != 'TRUE']

In [10]:
df_openFDA_NDC = df_openFDA_NDC.loc[df_openFDA_NDC['product_type'] == 'HUMAN PRESCRIPTION DRUG']

In [11]:
df_openFDA_NDC = df_openFDA_NDC.loc[df_openFDA_NDC['marketing_category'] != 'EMERGENCY USE AUTHORIZATION']
df_openFDA_NDC = df_openFDA_NDC.loc[df_openFDA_NDC['marketing_category'] != 'UNAPPROVED DRUG FOR USE IN DRUG SHORTAGE']
df_openFDA_NDC = df_openFDA_NDC.loc[df_openFDA_NDC['marketing_category'] != 'UNAPPROVED DRUG OTHER']
df_openFDA_NDC = df_openFDA_NDC.loc[df_openFDA_NDC['marketing_category'] != 'UNAPPROVED MEDICAL GAS']
df_openFDA_NDC = df_openFDA_NDC.loc[df_openFDA_NDC['marketing_category'] != 'DRUG FOR FURTHER PROCESSING']
df_openFDA_NDC = df_openFDA_NDC.loc[df_openFDA_NDC['marketing_category'] != 'UNAPPROVED HOMEOPATHIC']

In [12]:
len(df_openFDA_NDC)

48846

Unpacking active ingredients from results

In [13]:
df_openFDA_NDC_active_ingredients = pd.DataFrame(df_openFDA_NDC['active_ingredients'])

df_openFDA_NDC_active_ingredients['org_index'] = df_openFDA_NDC_active_ingredients.index

df_openFDA_NDC_active_ingredients = df_openFDA_NDC_active_ingredients.dropna(subset = ['active_ingredients'])

df_openFDA_NDC_active_ingredients = df_openFDA_NDC_active_ingredients.explode('active_ingredients')

count = 0

for i, j in zip(df_openFDA_NDC_active_ingredients['active_ingredients'], df_openFDA_NDC_active_ingredients['org_index']): # normalizing product and adding id for merge
    
    count = count + 1
    
    df_temp = pd.DataFrame(pd.json_normalize(i))
    df_temp['org_index'] = j
    
    if count == 1:
        df_openFDA_NDC_active_ingredients_nor = df_temp
    else:
        df_openFDA_NDC_active_ingredients_nor = pd.concat([df_openFDA_NDC_active_ingredients_nor, df_temp])
        
df_openFDA_NDC_active_ingredients_nor = df_openFDA_NDC_active_ingredients_nor.set_index('org_index')

2022-03-22 08:49:49,787 - INFO     - NumExpr defaulting to 4 threads.


Merging the unpacked data with the orig. data 

In [14]:
df_openFDA_NDC = pd.merge(df_openFDA_NDC_active_ingredients_nor, df_openFDA_NDC,  left_index = True, right_index = True, how = 'right')

In [15]:
len(df_openFDA_NDC)

55363

Removing unique ids without a name or strength

In [16]:
df_openFDA_NDC = df_openFDA_NDC.dropna(subset=['name', 'strength'])

In [17]:
len(df_openFDA_NDC)

54485

Unpacking is original packager from openfda under results

In [ ]:
df_openFDA_NDC_openFDA_org_pack = pd.DataFrame(df_openFDA_NDC['openfda'])

df_openFDA_NDC_openFDA_org_pack['org_index'] = df_openFDA_NDC_openFDA_org_pack.index

count = 0

for i, j in zip(df_openFDA_NDC_openFDA_org_pack['openfda'], df_openFDA_NDC_openFDA_org_pack['org_index']): # normalizing product and adding id for merge
    
    count = count + 1
    
    df_temp = pd.DataFrame(pd.json_normalize(i))
    df_temp['org_index'] = j
    
    if count == 1:
        df_openFDA_NDC_openFDA_org_pack_nor = df_temp
    else:
        df_openFDA_NDC_openFDA_org_pack_nor = pd.concat([df_openFDA_NDC_openFDA_org_pack_nor, df_temp])
        
df_openFDA_NDC_openFDA_org_pack_nor = df_openFDA_NDC_openFDA_org_pack_nor.set_index('org_index')

df_openFDA_NDC_openFDA_org_pack_nor = pd.DataFrame(df_openFDA_NDC_openFDA_org_pack_nor['is_original_packager'])

df_openFDA_NDC_openFDA_org_pack_nor = df_openFDA_NDC_openFDA_org_pack_nor.explode('is_original_packager')

df_openFDA_NDC_openFDA_org_pack_nor = df_openFDA_NDC_openFDA_org_pack_nor.dropna()

Merging the unpacked data with the orig. data

In [ ]:
df_openFDA_NDC = pd.merge(df_openFDA_NDC_openFDA_org_pack_nor, df_openFDA_NDC,  left_index = True, right_index = True, how = 'right')

In [ ]:
len(df_openFDA_NDC)

### Data Manipulation

Splitting ndc into labeler and product code

In [ ]:
df_temp = df_openFDA_NDC['product_ndc'].str.split("-", n = 1, expand = True)
df_openFDA_NDC['labeler_code']= df_temp[0]
df_openFDA_NDC['labeler_code'] = pd.to_numeric(df_openFDA_NDC['labeler_code'])
df_openFDA_NDC['product_code']= df_temp[1]
df_openFDA_NDC['product_code'] = pd.to_numeric(df_openFDA_NDC['product_code'])

Clearing route for characters that are not a letters or numbers

In [ ]:
df_openFDA_NDC['route'] = df_openFDA_NDC['route'].astype(str).apply(lambda x: re.sub('[^A-Za-z0-9]+', '', x)) 

Creating an unique id

In [ ]:
df_openFDA_NDC['generic_name'] = df_openFDA_NDC['name'].str.strip()
df_openFDA_NDC['dosage_form'] = df_openFDA_NDC['dosage_form'].str.strip()
df_openFDA_NDC['route'] = df_openFDA_NDC['route'].str.strip()
df_openFDA_NDC['strength'] = df_openFDA_NDC['strength'].str.strip()

df_openFDA_NDC['unique_id'] = df_openFDA_NDC['name'].astype(str) + '-' + df_openFDA_NDC['dosage_form'].astype(str) + '-' + df_openFDA_NDC['route'].astype(str) + '-' + df_openFDA_NDC['strength'].astype(str)
df_openFDA_NDC['unique_id'] = df_openFDA_NDC['unique_id'].str.lower()

Removing duplicates based on unique id, labeler and product code

In [ ]:
df_openFDA_NDC = df_openFDA_NDC.drop_duplicates(subset = ['unique_id', 'labeler_code', 'product_code'])
print(len(df_openFDA_NDC))

Adding false with unique ids that are not org. packed

In [ ]:
df_openFDA_NDC['is_original_packager'] = df_openFDA_NDC['is_original_packager'].fillna(False)

Removing unique ids without a name, strength or pharm_class

In [ ]:
df_openFDA_NDC = df_openFDA_NDC.dropna(subset=['name', 'strength', 'pharm_class'])

# Downloading output

In [ ]:
df_openFDA_NDC.to_csv('raw_openFDA_NDC_data.csv', sep = '~', index = False)